# Exercise 3

task description

In [1]:
import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf

In [2]:
! if ! [[ -f data/ner/gmb.csv ]]; then curl https://nilsreiter.de/assets/2020-08-31-deep-learning/ner/gmb.csv > data/ner/gmb.csv; fi

In [3]:
data = pd.read_csv("data/ner/gmb.csv",encoding = 'latin1')
data = data.fillna(method = 'ffill')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [4]:
class Get_sentence(object):
    def __init__(self,data):
        self.n_sent=1
        self.data = data
        agg_func = lambda s:[(w,p,t) for w,p,t in zip(s["Word"].values.tolist(),
                                                     s["POS"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [5]:
getter = Get_sentence(data)
sentence = getter.sentences
sentence[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [6]:
words = list(set(data["Word"].values))
words.append("ENDPAD")

words_tag = list(set(data["Tag"].values))

word_idx = {w : i+1 for i ,w in enumerate(words)}
tag_idx =  {t : i for i ,t in enumerate(words_tag)}

num_words = len(words)
num_words_tag = len(words_tag)

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 50
X = [[word_idx[w[0]] for w in s] for s in sentence]
X = pad_sequences(maxlen = max_len,sequences = X,padding = 'post',value = num_words-1)
y = [[tag_idx[w[2]] for w in s] for s in sentence]
y = pad_sequences(maxlen = max_len,sequences = y,padding = 'post',value = tag_idx['O'])
y = [to_categorical(i,num_classes = num_words_tag) for i in  y]

In [8]:
X[1:10]

array([[26923, 19139, 24867, 23709, 29041,  8430, 27858, 30155,  8430,
        29559, 28627, 33672, 12988, 31283, 28807, 17028, 28709, 16229,
        15584, 30917, 25100, 13758, 22296,  2732,  7293, 35178, 35178,
        35178, 35178, 35178, 35178, 35178, 35178, 35178, 35178, 35178,
        35178, 35178, 35178, 35178, 35178, 35178, 35178, 35178, 35178,
        35178, 35178, 35178, 35178, 35178],
       [11953,  6169, 23248, 30220, 29078,  7065, 27463, 31283, 33979,
         4527, 15710, 28709, 25904,  4095,    81, 10602, 11036,  5071,
         8430,  2453,  5702,  8430, 32447, 15584,  4987,   157, 26057,
        27463,  6749, 34361, 32053,  7293, 35178, 35178, 35178, 35178,
        35178, 35178, 35178, 35178, 35178, 35178, 35178, 35178, 35178,
        35178, 35178, 35178, 35178, 35178],
       [13702, 13477, 16229, 18119, 20228, 22352, 14426,  2273, 14172,
         8509,  7293, 35178, 35178, 35178, 35178, 35178, 35178, 35178,
        35178, 35178, 35178, 35178, 35178, 35178, 35178, 351

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state=1)

In [12]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense,SimpleRNN
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional
from tensorflow.python.keras import models

In [24]:
model = models.Sequential()
model.add(Input(shape = (max_len,)))
model.add(Embedding(input_dim = num_words, output_dim = 20, input_length = max_len))
model.add(SpatialDropout1D(0.1))
model.add(SimpleRNN(units=100, return_sequences = True, recurrent_dropout=0.1))
#model.add(Bidirectional(LSTM(units=100,return_sequences = True, recurrent_dropout = 0.1)))
model.add(TimeDistributed(Dense(num_words_tag,activation = 'softmax')))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 20)            703580    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 50, 20)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 50, 100)           12100     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 50, 17)            1717      
Total params: 717,397
Trainable params: 717,397
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 

history = model.fit(
    x_train,np.array(y_train),
    batch_size = 64,
    epochs = 3,
    verbose =1
)

Epoch 1/3
675/675 [==============================] - 11s 16ms/step - loss: 0.2854 - accuracy: 0.9438
Epoch 2/3
675/675 [==============================] - 11s 16ms/step - loss: 0.0740 - accuracy: 0.9796
Epoch 3/3
675/675 [==============================] - 11s 17ms/step - loss: 0.0538 - accuracy: 0.9841


In [20]:
model.evaluate(x_test,np.array(y_test))

150/150 [==============================] - 0s 3ms/step - loss: 0.0567 - accuracy: 0.9830


[0.05665678158402443, 0.9830400347709656]

TODO: Evaluation scores per class

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

Y_test = np.argmax(y_test, axis=1)

y_pred = np.argmax(model.predict(x_test), axis=-1)

#classification_report(Y_test, y_pred)
y_pred

In [ ]:
Y_test

## Challenge


In [ ]:
! curl https://nilsreiter.de/assets/2020-08-31-deep-learning/ner/challenge.wb.csv > data/ner/challenge.wb.csv

In [ ]:
! curl https://nilsreiter.de/assets/2020-08-31-deep-learning/ner/challenge.bc.csv > data/ner/challenge.bc.csv

In [ ]:
! curl https://nilsreiter.de/assets/2020-08-31-deep-learning/ner/challenge.nw.csv > data/ner/challenge.nw.csv

In [ ]:
challenge = pd.read_csv("data/ner/challenge.wb.csv", header = 0, names=["Sentence #","Word","POS","Tag"])
challenge.head()


In [ ]:
getter = Get_sentence(challenge)
sentence = getter.sentences
sentence[0]

In [ ]:
max_len = 50
x_challenge = [[word_idx.get(w[0],0) for w in s] for s in sentence]
x_challenge = pad_sequences(maxlen = max_len,sequences = x_challenge,padding = 'post',value = num_words-1)
y_challenge = [[tag_idx[w[2]] for w in s] for s in sentence]
y_challenge = pad_sequences(maxlen = max_len,sequences = y_challenge,padding =
                        'post',value = tag_idx['O'])
y_challenge = [to_categorical(i,num_classes = num_words_tag) for i in  y_challenge]

In [ ]:
y_challenge[1:10]

In [ ]:
model.evaluate(x_challenge, np.array(y_challenge))